### Imports

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import scipy.signal as sg
import matplotlib.pyplot as plt
from neuropy import plotting
import subjects

### Generate test dataset for sleep scoring

In [ ]:
from pathlib import Path

sess = subjects.nsd.ratUday2[0]

eeg = []
for state_name in ["AW", "QW", "REM", "NREM"]:
    states = sess.brainstates[state_name].duration_slice(300)[0]
    start, stop = states.starts, states.stops
    print(stop - start)
    eeg.append(sess.eegfile.get_signal(t_start=start, t_stop=stop).traces)
eeg = np.hstack(eeg)

dest_dir = Path("/data/Clustering/sessions/Simulated/RatSim1/")
dest_file = dest_dir / "RatSim1.eeg"
dtype = sess.eegfile.dtype
n_data = eeg.shape[0] * eeg.shape[1]

# #Uncomment to write data
# data = np.memmap(dest_file, dtype=dtype, mode="w+", shape=n_data)
# data[:] = eeg.flatten("F")

### Generate PSD estimates for various states

In [ ]:
sess = subjects.sd.ratUday4[0]
fs = sess.eegfile.sampling_rate
psd = {}
for state_name in ["AW", "QW", "REM", "NREM"]:
    states = sess.brainstates[state_name].duration_slice(200)[0]
    start, stop = states.starts[0], states.stops[0]
    lfp = sess.eegfile.get_signal(channel_indx=86, t_start=start, t_stop=stop).traces[0]
    freq, psd_state = sg.welch(lfp, fs=fs, nperseg=2 * fs, noverlap=fs)
    indx = freq <= 40
    psd[state_name] = psd_state[indx]
psd["freq"] = freq[indx]
# np.save("/data/Clustering/sessions/psd_states.npy", psd)

In [ ]:
_, ax = plt.subplots()

for i, state_name in enumerate(["AW", "QW", "REM", "NREM"]):
    ax.plot(psd["freq"], psd[state_name], label=state_name)

# ax.set_xscale('log')
ax.set_yscale("log")
ax.legend()
# ax.set_xlim(0, 100)

In [ ]:
from neuropy.utils.signal_process import filter_sig, FourierSg, WaveletSg


lfp = sess.eegfile.get_signal(102)
# filt_lfp = filter_sig.highpass(lfp, 60)
lfp_spect = FourierSg(lfp, norm_sig=True)
# lfp_spect = WaveletSg(lfp, freqs=np.arange(0.5, 16, 0.5))
# zsc_sig = stats.zscore(lfp.traces[0])

In [ ]:
_, ax = plt.subplots()

ax.plot(lfp.time, np.abs(stats.zscore(lfp.traces[0])), ".", markersize=1)

In [108]:
_, ax = plt.subplots()

ax.plot(np.abs(np.diff(lfp.traces[0], 10)))

In [ ]:
a = np.arange(10000).astype(float)
a[30] = np.nan

spect = sg.spectrogram(a)[2]

_, ax = plt.subplots()
ax.pcolormesh(spect)

In [ ]:
from neuropy.utils.mathutil import min_max_scaler
import matplotlib.colors as mpl_colors

_, ax = plt.subplots()
# ax.plot(lfp.time, zsc_sig)
# ax.pcolormesh(lfp_spect.time, lfp_spect.freqs, lfp_spect.traces, vmin=0.1, vmax=0.2)
spect_slice = lfp_spect.freq_slice(0, 45)
spect_traces = spect_slice.traces
spect_trace_mean = (
    m := spect_slice.time_slice(13500, 15000).traces.mean(axis=1)
) / m.sum()
spect_traces_scaled = spect_traces / spect_traces.sum(axis=0, keepdims=True)
diff_trace = np.abs(spect_traces_scaled - spect_trace_mean[:, np.newaxis]).max(axis=0)
extent = [
    spect_slice.time[0],
    spect_slice.time[-1],
    spect_slice.freqs[0],
    spect_slice.freqs[-1],
]

ax.imshow(
    (z := spect_traces_scaled),
    aspect="auto",
    # norm=mpl_colors.LogNorm(vmin=z.min()*4, vmax=z.max() / 3),
    # norm=mpl_colors.PowerNorm(gamma=0.3),
    vmax=0.5,
    origin="lower",
    extent=extent,
)

# ax.plot(diff_trace)
# ax.set_ylim(0, 40)
# ax.plot(lfp.time[:-1], stats.zscore(np.diff(lfp.traces[0])))
# ax.plot(stats.zscore(np.gradient(lfp.traces[0])))

# ax.plot(lfp.time, filt_lfp.traces[0])

In [ ]:
_, ax = plt.subplots()
ax.plot(spect_traces_scaled[10:20, :].mean(axis=0))

In [ ]:
from scipy import interpolate
from neuropy.analyses.brainstates import get_artifact_indices, interpolate_indices


mean_psd = lfp_spect.traces.mean(axis=0)
artifact_indices = get_artifact_indices(mean_psd, thresh=0.22)
zsc_mean_psd = stats.zscore(mean_psd)
zsc_mean_psd_interp = interpolate_indices(zsc_mean_psd, artifact_indices)
# delta_interp = interpolate_indices(lfp_spect.delta, artifact_indices)
# theta_interp = interpolate_indices(lfp_spect.theta, artifact_indices)
# # peaks, peaks_prop = sg.find_peaks(
#     np.where(zsc_mean_psd > 0, zsc_mean_psd, 0), height=3, prominence=1, plateau_size=1
# )
# left_edge = peaks_prop["left_edges"]
# right_edge = peaks_prop["right_edges"]
# left_base = peaks_prop["left_bases"]
# right_base = peaks_prop["right_bases"]
# nan_indx = np.unique(
#     np.concatenate([np.arange(a, b + 1) for a, b in zip(left_base, right_base)])
# )


# art_ind = zsc_mean_psd > 3
# # nan_mean_psd = np.where(zsc_mean_psd > 3, np.nan, zsc_mean_psd)
# nan_mean_psd = zsc_mean_psd.copy()
# nan_mean_psd[nan_indx] = np.nan
# interp_df = pd.DataFrame(nan_mean_psd).interpolate(method="cubic")[0].to_numpy()
# interp_delta = pd.DataFrame(nan_mean_psd).interpolate(method="cubic")[0].to_numpy()
# interp_data = interpolate.interp1d(
#     lfp_spect.time[~art_ind],
#     zsc_mean_psd[~art_ind],
#     bounds_error=False,
#     fill_value="extrapolate",
# )
# ax.plot(stats.zscore(lfp_spect.traces[1]))

_, ax = plt.subplots(2, 1, sharey=False, sharex=True)
ax[0].plot(zsc_mean_psd, ".", markersize=1)
# ax[0].plot(zsc_mean_psd_interp)
# ax[1].plot(lfp_spect.theta)
# ax[1].plot(theta_interp / delta_interp, ".", markersize=1)
# ax.plot(zsc_mean_psd_interp)
# ax.plot(, "k*")
# ax.plot(left_base, zsc_mean_psd[left_base], "r*")
# ax.plot(right_base, zsc_mean_psd[right_base] + 0.5, "g*")
# ax.plot(interp_data(lfp_spect.time))

In [ ]:
from scipy.ndimage import gaussian_filter1d

score = stats.scoreatpercentile(zsc_mean_psd, 99.5)
_, ax = plt.subplots()
ax.plot(gaussian_filter1d(zsc_mean_psd, 5 / 0.5), ".", markersize=1)
ax.axhline(score, color="k")

In [ ]:
from neuropy.utils.signal_process import hilbertfast

_, ax = plt.subplots()

# ax.scatter(lfp_spect.theta, lfp_spect.delta, s=2)

# hil_amp = np.abs(hilbertfast(zsc_mean_psd))
# ax.plot(hil_amp)
bins = np.linspace(-1.5, 5, 200)
# bins = np.linspace(0,0.01,200)
hist_zsc_psd = np.histogram(zsc_mean_psd, bins=bins)[0]
# ax.plot(zsc_mean_psd)
ax.plot(hist_zsc_psd / zsc_mean_psd.size)

In [ ]:
stats.scoreatpercentile(zsc_mean_psd, 99.99)

In [ ]:
_, ax = plt.subplots(2, 1, sharex=True, sharey=True)
# ax.plot(zsc_mean_psd)
ax[0].plot(np.abs(stats.zscore(np.diff(mean_psd))))
ax[1].plot(zsc_mean_psd)

In [ ]:
a = np.zeros(1000)
a[np.random.choice(1000, 100)] = 100
sig = np.abs(stats.zscore(sess.eegfile.get_signal(102).traces[0]))
perc = stats.scoreatpercentile(sig, 99)
perc

In [ ]:
stats.scoreatpercentile(sig, 99.5)

In [ ]:
_, ax = plt.subplots()

bins = np.linspace(np.min(zsc_mean_psd), np.max(zsc_mean_psd), 100)
hist_psd = np.histogram(zsc_mean_psd, bins=bins)[0]
ax.plot(hist_psd)
# ax.set_yscale('log')
# ax.set_xscale('log')

In [ ]:
_, ax = plt.subplots()
ax.plot(sig)
ax.axhline(4.5)